In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()
output_null = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://portal.thepartshq.com/rest/V1/marmon/product/details/sku/' + input_.loc[a, 'Part_Number'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                if json_data[0]['_value'] == None:
                    crawler_status = 'null'
                    break
                
                # = = = = = = = = = = = = = = =
                
                part_type = '' if json_data[0]['_value']['part_type'] == None else json_data[0]['_value']['part_type'].strip()

                # = = = = = = = = = = = = = = =
                
                list_vehicle = [] if json_data[0]['_value']['buyersguide'] == '1' else json_data[0]['_value']['buyersguide'][1]['app']
                list_vehicle = list_vehicle if type(list_vehicle) == list else [list_vehicle]
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for vehicle in list_vehicle:
                    if vehicle['make'].strip() not in dict_vehicle:
                        dict_vehicle[vehicle['make'].strip()] = {vehicle['model'].strip(): {vehicle['engine'].strip(): [vehicle['year'].strip()]}}
                    else:
                        if vehicle['model'].strip() not in dict_vehicle[vehicle['make'].strip()]:
                            dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()] = {vehicle['engine'].strip(): [vehicle['year'].strip()]}
                        else:
                            if vehicle['engine'].strip() not in dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()]:
                                dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()][vehicle['engine'].strip()] = [vehicle['year'].strip()]
                            else:
                                dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()][vehicle['engine'].strip()].append(vehicle['year'].strip())

                    dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()][vehicle['engine'].strip()] = sorted(list(set(dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()][vehicle['engine'].strip()])))

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_year = []
                        for engine in dict_vehicle[make][model]:
                            list_year += dict_vehicle[make][model][engine]
                        list_year = sorted(list(set(list_year)))

                        list_vehicle.append(make + ' ' + model + ' ' + ','.join(list_year))

                vehicle_1 = '\n'.join(sorted(list_vehicle))

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        for engine in dict_vehicle[make][model]:
                            if engine == '':
                                list_vehicle.append(make + ' ' + model + ' ' + ','.join(list_year))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + ','.join(list_year) + ' ' + engine)

                vehicle_2 = '\n'.join(sorted(list_vehicle))

                # = = = = = = = = = = = = = = =
                
                src = json_data[0]['_value']['images']['image']['id'].strip() if type(json_data[0]['_value']['images']['image']) == dict else json_data[0]['_value']['images']['image'][0]['id'].strip()
                    
                # = = = = = = = = = = = = = = =
                
                list_attribute = json_data[0]['_value']['Attributes']['partattributes'] if 'Attributes' in json_data[0]['_value'] else []
                list_attribute = list_attribute if type(list_attribute) == list else [list_attribute]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Part_Number': input_.loc[a, 'Part_Number'],
                                         'Part_Type': part_type,
                                         'Vehicle_1': vehicle_1,
                                         'Vehicle_2': vehicle_2,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for attribute in list_attribute:
                    df_temp.loc[0, attribute['attribute'].strip()] = attribute['value'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'null':
            df_temp = pd.DataFrame([{'No.': input_.loc[a, 'No.'],
                                     'Part_Number': int(input_.loc[a, 'Part_Number']),
                                     'Url': input_.loc[a, 'Url']}])

            output_null = pd.concat([output_null, df_temp], ignore_index=True).fillna('')
    
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
if len(output_null) != 0:
    output_null = output_null.drop_duplicates(ignore_index=True)
    output_null = output_null.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_null.to_excel('./part_null.xlsx', index=False)
    print('爬虫存在null')
    print()
print('Done ~')

总数量：1266

[ok] - A2163
[尝试次数：1] - [剩余数量：1246] - [当前时间：10:07:58]

[ok] - A2162
[尝试次数：1] - [剩余数量：1245] - [当前时间：10:07:59]

[ok] - A2039
[尝试次数：1] - [剩余数量：1244] - [当前时间：10:07:59]

[ok] - A2217
[尝试次数：1] - [剩余数量：1243] - [当前时间：10:07:59]

[ok] - A2201
[尝试次数：1] - [剩余数量：1242] - [当前时间：10:07:59]

[ok] - A2236
[尝试次数：1] - [剩余数量：1241] - [当前时间：10:08:00]

[ok] - A2348
[尝试次数：1] - [剩余数量：1240] - [当前时间：10:08:01]

[ok] - A2073
[尝试次数：1] - [剩余数量：1239] - [当前时间：10:08:01]

[ok] - A2306
[尝试次数：1] - [剩余数量：1238] - [当前时间：10:08:02]

[ok] - A2299
[尝试次数：1] - [剩余数量：1237] - [当前时间：10:08:02]

[ok] - A2349
[尝试次数：1] - [剩余数量：1236] - [当前时间：10:08:02]

[ok] - A2320
[尝试次数：1] - [剩余数量：1235] - [当前时间：10:08:03]

[ok] - A2327
[尝试次数：1] - [剩余数量：1234] - [当前时间：10:08:03]

[ok] - A2374
[尝试次数：1] - [剩余数量：1233] - [当前时间：10:08:03]

[ok] - A2388
[尝试次数：1] - [剩余数量：1232] - [当前时间：10:08:04]

[ok] - A2379
[尝试次数：1] - [剩余数量：1231] - [当前时间：10:08:04]

[ok] - A2253
[尝试次数：1] - [剩余数量：1230] - [当前时间：10:08:04]

[ok] - A2394
[尝试次数：1] - [剩余数量：1229] - [当前时间：10:08:05]



[ok] - A2982
[尝试次数：1] - [剩余数量：1096] - [当前时间：10:09:21]

[ok] - A2986
[尝试次数：1] - [剩余数量：1095] - [当前时间：10:09:21]

[ok] - A2988
[尝试次数：1] - [剩余数量：1094] - [当前时间：10:09:22]

[ok] - A3041
[尝试次数：1] - [剩余数量：1093] - [当前时间：10:09:22]

[ok] - A3032
[尝试次数：1] - [剩余数量：1092] - [当前时间：10:09:23]

[ok] - A2924
[尝试次数：1] - [剩余数量：1091] - [当前时间：10:09:23]

[ok] - A3052
[尝试次数：1] - [剩余数量：1090] - [当前时间：10:09:23]

[ok] - A2817
[尝试次数：1] - [剩余数量：1089] - [当前时间：10:09:24]

[ok] - A3054
[尝试次数：1] - [剩余数量：1088] - [当前时间：10:09:25]

[ok] - A3081
[尝试次数：1] - [剩余数量：1087] - [当前时间：10:09:25]

[ok] - A3027
[尝试次数：1] - [剩余数量：1086] - [当前时间：10:09:25]

[ok] - A3082
[尝试次数：1] - [剩余数量：1085] - [当前时间：10:09:26]

[ok] - A4000
[尝试次数：1] - [剩余数量：1084] - [当前时间：10:09:26]

[ok] - A4028
[尝试次数：1] - [剩余数量：1083] - [当前时间：10:09:27]

[ok] - A3065
[尝试次数：1] - [剩余数量：1082] - [当前时间：10:09:27]

[ok] - A3026
[尝试次数：1] - [剩余数量：1081] - [当前时间：10:09:27]

[ok] - A4004
[尝试次数：1] - [剩余数量：1080] - [当前时间：10:09:28]

[ok] - A3069
[尝试次数：1] - [剩余数量：1079] - [当前时间：10:09:29]

[ok] - A28

[ok] - A4928
[尝试次数：1] - [剩余数量：945] - [当前时间：10:10:36]

[ok] - A4528
[尝试次数：2] - [剩余数量：944] - [当前时间：10:10:36]

[ok] - A4932
[尝试次数：1] - [剩余数量：943] - [当前时间：10:10:37]

[ok] - A4586
[尝试次数：1] - [剩余数量：942] - [当前时间：10:10:37]

[ok] - A4921
[尝试次数：1] - [剩余数量：941] - [当前时间：10:10:38]

[ok] - A4933
[尝试次数：1] - [剩余数量：940] - [当前时间：10:10:39]

[ok] - A4920
[尝试次数：1] - [剩余数量：939] - [当前时间：10:10:39]

[ok] - A4939
[尝试次数：1] - [剩余数量：938] - [当前时间：10:10:39]

[ok] - A4936
[尝试次数：1] - [剩余数量：937] - [当前时间：10:10:39]

[ok] - A5132
[尝试次数：1] - [剩余数量：936] - [当前时间：10:10:41]

[ok] - A4923
[尝试次数：1] - [剩余数量：935] - [当前时间：10:10:41]

[ok] - A5209
[尝试次数：1] - [剩余数量：934] - [当前时间：10:10:41]

[ok] - A5212
[尝试次数：1] - [剩余数量：933] - [当前时间：10:10:41]

[ok] - A2893
[尝试次数：2] - [剩余数量：932] - [当前时间：10:10:41]

[ok] - A5219B
[尝试次数：1] - [剩余数量：931] - [当前时间：10:10:42]

[ok] - A5225
[尝试次数：1] - [剩余数量：930] - [当前时间：10:10:43]

[ok] - A5123
[尝试次数：1] - [剩余数量：929] - [当前时间：10:10:43]

[ok] - A5223
[尝试次数：1] - [剩余数量：928] - [当前时间：10:10:43]

[ok] - A4593
[尝试次数：1] - [剩余

[ok] - A5655
[尝试次数：1] - [剩余数量：793] - [当前时间：10:11:42]

[ok] - A5711
[尝试次数：1] - [剩余数量：792] - [当前时间：10:11:42]

[ok] - A5620
[尝试次数：2] - [剩余数量：791] - [当前时间：10:11:42]

[ok] - A5696
[尝试次数：1] - [剩余数量：790] - [当前时间：10:11:43]

[ok] - A5721
[尝试次数：1] - [剩余数量：789] - [当前时间：10:11:43]

[ok] - A5632
[尝试次数：1] - [剩余数量：788] - [当前时间：10:11:43]

[ok] - A5683
[尝试次数：1] - [剩余数量：787] - [当前时间：10:11:43]

[ok] - A5727
[尝试次数：1] - [剩余数量：786] - [当前时间：10:11:44]

[ok] - A5713
[尝试次数：1] - [剩余数量：785] - [当前时间：10:11:44]

[ok] - A5643
[尝试次数：1] - [剩余数量：784] - [当前时间：10:11:44]

[ok] - A5726
[尝试次数：1] - [剩余数量：783] - [当前时间：10:11:44]

[ok] - A5733
[尝试次数：1] - [剩余数量：782] - [当前时间：10:11:44]

[ok] - A5734
[尝试次数：1] - [剩余数量：781] - [当前时间：10:11:44]

[ok] - A5584
[尝试次数：2] - [剩余数量：780] - [当前时间：10:11:45]

[ok] - A5728
[尝试次数：1] - [剩余数量：779] - [当前时间：10:11:45]

[ok] - A5742
[尝试次数：1] - [剩余数量：778] - [当前时间：10:11:46]

[ok] - A5676
[尝试次数：1] - [剩余数量：777] - [当前时间：10:11:46]

[ok] - A5737
[尝试次数：2] - [剩余数量：776] - [当前时间：10:11:46]

[ok] - A5748
[尝试次数：2] - [剩余数

[ok] - A6248
[尝试次数：1] - [剩余数量：641] - [当前时间：10:12:47]

[ok] - A6294
[尝试次数：1] - [剩余数量：640] - [当前时间：10:12:48]

[ok] - A62094
[尝试次数：1] - [剩余数量：639] - [当前时间：10:12:49]

[ok] - A6320
[尝试次数：1] - [剩余数量：638] - [当前时间：10:12:51]

[ok] - A6334
[尝试次数：1] - [剩余数量：637] - [当前时间：10:12:51]

[ok] - A6329
[尝试次数：1] - [剩余数量：636] - [当前时间：10:12:51]

[ok] - A6311
[尝试次数：1] - [剩余数量：635] - [当前时间：10:12:53]

[ok] - A6352
[尝试次数：1] - [剩余数量：634] - [当前时间：10:12:53]

[ok] - A6256
[尝试次数：2] - [剩余数量：633] - [当前时间：10:12:54]

[ok] - A6339
[尝试次数：1] - [剩余数量：632] - [当前时间：10:12:54]

[ok] - A6312
[尝试次数：1] - [剩余数量：631] - [当前时间：10:12:55]

[ok] - A6372
[尝试次数：1] - [剩余数量：630] - [当前时间：10:12:55]

[ok] - A6360
[尝试次数：1] - [剩余数量：629] - [当前时间：10:12:56]

[ok] - A6373
[尝试次数：1] - [剩余数量：628] - [当前时间：10:12:56]

[ok] - A62018
[尝试次数：1] - [剩余数量：627] - [当前时间：10:12:56]

[ok] - A6286
[尝试次数：2] - [剩余数量：626] - [当前时间：10:12:57]

[ok] - A6391
[尝试次数：1] - [剩余数量：625] - [当前时间：10:12:58]

[ok] - A6403
[尝试次数：1] - [剩余数量：624] - [当前时间：10:12:58]

[ok] - A6392
[尝试次数：1] - [剩

[ok] - A6694
[尝试次数：2] - [剩余数量：490] - [当前时间：10:14:03]

[ok] - A6796
[尝试次数：1] - [剩余数量：489] - [当前时间：10:14:04]

[ok] - A6839
[尝试次数：1] - [剩余数量：488] - [当前时间：10:14:04]

[ok] - A6843
[尝试次数：5] - [剩余数量：487] - [当前时间：10:14:05]

[ok] - A6825
[尝试次数：1] - [剩余数量：486] - [当前时间：10:14:05]

[ok] - A65060
[尝试次数：1] - [剩余数量：485] - [当前时间：10:14:06]

[ok] - A6838
[尝试次数：4] - [剩余数量：484] - [当前时间：10:14:06]

[ok] - A65094
[尝试次数：2] - [剩余数量：483] - [当前时间：10:14:06]

[ok] - A6905
[尝试次数：1] - [剩余数量：482] - [当前时间：10:14:07]

[ok] - A6791
[尝试次数：1] - [剩余数量：481] - [当前时间：10:14:07]

[ok] - A6901
[尝试次数：4] - [剩余数量：480] - [当前时间：10:14:07]

[ok] - A6907
[尝试次数：3] - [剩余数量：479] - [当前时间：10:14:08]

[ok] - A6919
[尝试次数：2] - [剩余数量：478] - [当前时间：10:14:08]

[ok] - A6933
[尝试次数：2] - [剩余数量：477] - [当前时间：10:14:10]

[ok] - A6708
[尝试次数：1] - [剩余数量：476] - [当前时间：10:14:10]

[ok] - A6924
[尝试次数：6] - [剩余数量：475] - [当前时间：10:14:10]

[ok] - A6923
[尝试次数：7] - [剩余数量：474] - [当前时间：10:14:11]

[ok] - A6650
[尝试次数：3] - [剩余数量：473] - [当前时间：10:14:12]

[ok] - A6938
[尝试次数：3] - [剩

[ok] - A7359
[尝试次数：1] - [剩余数量：339] - [当前时间：10:15:33]

[ok] - A7330
[尝试次数：1] - [剩余数量：338] - [当前时间：10:15:33]

[ok] - A7375
[尝试次数：1] - [剩余数量：337] - [当前时间：10:15:33]

[ok] - A7372
[尝试次数：1] - [剩余数量：336] - [当前时间：10:15:33]

[ok] - A7387
[尝试次数：1] - [剩余数量：335] - [当前时间：10:15:35]

[ok] - A7366
[尝试次数：1] - [剩余数量：334] - [当前时间：10:15:36]

[ok] - A7357
[尝试次数：1] - [剩余数量：333] - [当前时间：10:15:36]

[ok] - A7350
[尝试次数：1] - [剩余数量：332] - [当前时间：10:15:37]

[ok] - AM001
[尝试次数：1] - [剩余数量：331] - [当前时间：10:15:37]

[ok] - A70012
[尝试次数：9] - [剩余数量：330] - [当前时间：10:15:38]

[ok] - A7392
[尝试次数：1] - [剩余数量：329] - [当前时间：10:15:38]

[ok] - A7300
[尝试次数：2] - [剩余数量：328] - [当前时间：10:15:38]

[ok] - AM006
[尝试次数：1] - [剩余数量：327] - [当前时间：10:15:39]

[ok] - AM014
[尝试次数：1] - [剩余数量：326] - [当前时间：10:15:40]

[ok] - AM008
[尝试次数：1] - [剩余数量：325] - [当前时间：10:15:40]

[ok] - AM015
[尝试次数：1] - [剩余数量：324] - [当前时间：10:15:40]

[ok] - A7390
[尝试次数：1] - [剩余数量：323] - [当前时间：10:15:40]

[ok] - AM018
[尝试次数：1] - [剩余数量：322] - [当前时间：10:15:40]

[ok] - A7322
[尝试次数：1] - [剩余

[ok] - A2984
[尝试次数：2] - [剩余数量：187] - [当前时间：10:17:06]

[ok] - A4359
[尝试次数：1] - [剩余数量：186] - [当前时间：10:17:06]

[ok] - A2771
[尝试次数：1] - [剩余数量：185] - [当前时间：10:17:06]

[ok] - A2428
[尝试次数：2] - [剩余数量：184] - [当前时间：10:17:06]

[ok] - AM174
[尝试次数：2] - [剩余数量：183] - [当前时间：10:17:07]

[ok] - A2922
[尝试次数：2] - [剩余数量：182] - [当前时间：10:17:07]

[ok] - A2977
[尝试次数：1] - [剩余数量：181] - [当前时间：10:17:07]

[ok] - A4558
[尝试次数：1] - [剩余数量：180] - [当前时间：10:17:07]

[ok] - A4357
[尝试次数：1] - [剩余数量：179] - [当前时间：10:17:08]

[ok] - A4596
[尝试次数：1] - [剩余数量：178] - [当前时间：10:17:09]

[ok] - A4581
[尝试次数：1] - [剩余数量：177] - [当前时间：10:17:09]

[ok] - A42006
[尝试次数：1] - [剩余数量：176] - [当前时间：10:17:09]

[ok] - A4626
[尝试次数：1] - [剩余数量：175] - [当前时间：10:17:09]

[ok] - A4033
[尝试次数：2] - [剩余数量：174] - [当前时间：10:17:10]

[ok] - A4641
[尝试次数：1] - [剩余数量：173] - [当前时间：10:17:12]

[ok] - A4655
[尝试次数：1] - [剩余数量：172] - [当前时间：10:17:12]

[ok] - A4706
[尝试次数：1] - [剩余数量：171] - [当前时间：10:17:13]

[ok] - A4513
[尝试次数：1] - [剩余数量：170] - [当前时间：10:17:14]

[ok] - A5034
[尝试次数：1] - [剩余

[ok] - A71006
[尝试次数：1] - [剩余数量：34] - [当前时间：10:18:07]

[ok] - A7248
[尝试次数：1] - [剩余数量：33] - [当前时间：10:18:08]

[ok] - A7135
[尝试次数：5] - [剩余数量：32] - [当前时间：10:18:09]

[ok] - A7267
[尝试次数：1] - [剩余数量：31] - [当前时间：10:18:09]

[ok] - A7226
[尝试次数：1] - [剩余数量：30] - [当前时间：10:18:09]

[ok] - A7279
[尝试次数：1] - [剩余数量：29] - [当前时间：10:18:11]

[ok] - A7287
[尝试次数：1] - [剩余数量：28] - [当前时间：10:18:12]

[ok] - A7231
[尝试次数：1] - [剩余数量：27] - [当前时间：10:18:14]

[ok] - A7293
[尝试次数：1] - [剩余数量：26] - [当前时间：10:18:14]

[ok] - A7284
[尝试次数：1] - [剩余数量：25] - [当前时间：10:18:14]

[ok] - A7309
[尝试次数：1] - [剩余数量：24] - [当前时间：10:18:15]

[ok] - A4597
[尝试次数：6] - [剩余数量：23] - [当前时间：10:18:16]

[ok] - A4342
[尝试次数：3] - [剩余数量：22] - [当前时间：10:18:16]

[ok] - A71084
[尝试次数：3] - [剩余数量：21] - [当前时间：10:18:16]

[ok] - A6568
[尝试次数：2] - [剩余数量：20] - [当前时间：10:18:17]

[ok] - AM048
[尝试次数：1] - [剩余数量：19] - [当前时间：10:18:17]

[ok] - AM013
[尝试次数：1] - [剩余数量：18] - [当前时间：10:18:18]

[ok] - A71079
[尝试次数：8] - [剩余数量：17] - [当前时间：10:18:19]

[ok] - AM057
[尝试次数：1] - [剩余数量：16] - [当前时间：1